In [57]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [58]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time



In [59]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(5)

In [60]:
# running the model through EMA workbench
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=50, policies=2)

[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 50 scenarios * 2 policies * 1 model(s) = 100 experiments
100%|████████████████████████████████████████| 100/100 [00:08<00:00, 11.13it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [61]:
# observing the simulation runs
experiments, outcomes = results
# print(outcomes.keys())

In [62]:
#latin hypercube
df_exp = experiments

In [63]:
df = pd.DataFrame()

for k in outcomes:
    test_list = []
    for i in range(len(experiments)):
        t = outcomes[k][i].sum()
        test_list.append(t)
    df[k]=test_list

In [64]:
import pickle

with open('df_outcomes.pkl','wb') as pickle_file:
    pickle.dump(df, pickle_file)

In [65]:
df_outcomes = pd.read_pickle('df_outcomes.pkl')

In [66]:
df_outcomes["A.1_Deaths_Cost"]=df_outcomes["A.1_Expected Number of Deaths"]*2700000
df_outcomes["A.2_Deaths_Cost"]=df_outcomes["A.2_Expected Number of Deaths"]*2700000
df_outcomes["A.3_Deaths_Cost"]=df_outcomes["A.3_Expected Number of Deaths"]*2700000
df_outcomes["A.4_Deaths_Cost"]=df_outcomes["A.4_Expected Number of Deaths"]*2700000
df_outcomes["A.5_Deaths_Cost"]=df_outcomes["A.5_Expected Number of Deaths"]*2700000

In [67]:
to_drop = ["A.1_Expected Number of Deaths","A.2_Expected Number of Deaths","A.3_Expected Number of Deaths","A.4_Expected Number of Deaths","A.5_Expected Number of Deaths"]
df_outcomes = df_outcomes.drop(to_drop, axis=1)
df_outcomes['total_cost[€]']= df_outcomes.sum(axis=1)
df_total_cost = df_outcomes['total_cost[€]']

In [72]:
results = pd.concat([df_exp, df_total_cost], axis=1)

In [70]:
results

,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat,scenario,policy,model,total_cost[€]
0,49,255.353394,10.0,0.857858,153.945334,1.5,0.082724,283.572124,10.0,0.144225,...,6,4,1,10,7,0,54,52,dikesnet,1.919999e+09
1,110,199.955250,10.0,0.259020,257.246386,1.5,0.208543,314.759439,1.0,0.642869,...,6,4,1,10,7,0,55,52,dikesnet,1.912118e+09
2,87,186.283319,1.0,0.188153,113.923406,10.0,0.775229,159.722035,1.5,0.626463,...,6,4,1,10,7,0,56,52,dikesnet,2.100863e+09
3,74,210.207264,10.0,0.825674,146.468051,1.0,0.698236,39.067007,10.0,0.715761,...,6,4,1,10,7,0,57,52,dikesnet,1.997800e+09
4,57,140.352566,1.5,0.219324,328.733967,1.0,0.505976,61.089220,10.0,0.034604,...,6,4,1,10,7,0,58,52,dikesnet,2.150337e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,44,330.800695,10.0,0.615837,61.855487,10.0,0.736271,97.374602,1.5,0.197463,...,0,10,9,0,0,2,99,53,dikesnet,1.632190e+09
96,95,217.455686,1.0,0.390092,342.671702,1.0,0.180443,70.767263,10.0,0.720447,...,0,10,9,0,0,2,100,53,dikesnet,1.643130e+09
97,126,103.662486,10.0,0.122901,128.440781,10.0,0.913998,308.535021,1.0,0.421243,...,0,10,9,0,0,2,101,53,dikesnet,1.632190e+09
98,92,323.485979,1.5,0.429788,286.929143,1.0,0.521217,104.429233,1.5,0.677923,...,0,10,9,0,0,2,102,53,dikesnet,1.632190e+09


In [11]:
# # defining specific policies
# # for example, policy 1 is about extra protection in upper boundary
# # policy 2 is about extra protection in lower boundary
# # policy 3 is extra protection in random locations
#
#
# def get_do_nothing_dict():
#     return {l.name: 0 for l in dike_model.levers}
#
#
# policies = [
#     Policy(
#         "policy 1",
#         **dict(
#             get_do_nothing_dict(),
#             **{"0_RfR 0": 1, "0_RfR 1": 1, "0_RfR 2": 1, "A.1_DikeIncrease 0": 5}
#         )
#     ),
#     Policy(
#         "policy 2",
#         **dict(
#             get_do_nothing_dict(),
#             **{"4_RfR 0": 1, "4_RfR 1": 1, "4_RfR 2": 1, "A.5_DikeIncrease 0": 5}
#         )
#     ),
#     Policy(
#         "policy 3",
#         **dict(
#             get_do_nothing_dict(),
#             **{"1_RfR 0": 1, "2_RfR 1": 1, "3_RfR 2": 1, "A.3_DikeIncrease 0": 5}
#         )
#     ),
# ]

In [12]:
# # pass the policies list to EMA workbench experiment runs
# n_scenarios = 100
# with MultiprocessingEvaluator(dike_model) as evaluator:
#     results = evaluator.perform_experiments(n_scenarios, policies)

[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 100 scenarios * 3 policies * 1 model(s) = 300 experiments
100%|████████████████████████████████████████| 300/300 [00:25<00:00, 11.67it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [13]:
# experiments, outcomes = results

In [14]:
# # only works because we have scalar outcomes
# pd.DataFrame(outcomes)

,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,5.397251e+07,0.000000,4.620660e+06,0.005443,4.152414e+07,0.090124,2.824868e+07,0.016040,5.315896e+07,0.056448,253800000.0,0.0
1,5.397251e+07,0.000000,9.875764e+07,0.096251,1.476749e+09,2.594713,1.061808e+07,0.004789,1.715375e+06,0.001558,253800000.0,0.0
2,5.397251e+07,0.000000,4.682906e+07,0.050271,2.386701e+08,0.473904,5.485795e+07,0.027997,0.000000e+00,0.000000,253800000.0,0.0
3,5.397251e+07,0.000000,5.559100e+07,0.050168,2.106536e+08,0.353636,0.000000e+00,0.000000,0.000000e+00,0.000000,253800000.0,0.0
4,5.397251e+07,0.000000,9.558758e+06,0.011308,1.141946e+09,2.315536,6.373503e+06,0.003377,0.000000e+00,0.000000,253800000.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
295,1.625123e+07,0.009414,4.625496e+07,0.037693,2.879840e+07,0.000000,1.218871e+07,0.005005,0.000000e+00,0.000000,369700000.0,0.0
296,1.341257e+09,0.837254,1.460801e+06,0.001369,2.879840e+07,0.000000,1.188652e+07,0.005491,0.000000e+00,0.000000,369700000.0,0.0
297,2.975870e+09,1.660996,7.115702e+07,0.057468,2.879840e+07,0.000000,2.301570e+07,0.009150,0.000000e+00,0.000000,369700000.0,0.0
298,0.000000e+00,0.000000,7.561023e+08,0.597678,2.879840e+07,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,369700000.0,0.0
